In [1]:
    import traitlets, ipywidgets, IPython, abc, collections, inspect, functools, typing
    ANNO, CTX, RTN = "__annotations__ @context return".split()
    

In [2]:
    class UnionMeta(abc.ABCMeta):
        def __getitem__(Union, object):
            if not isinstance(object, tuple): object = object,
            return Union(*object)
            try: return typing.Union[object]
            except (SyntaxError, TypeError): return Union(*object)
                
    class Union(metaclass=UnionMeta):
        def __init__(Union, *args): Union.__args__ = args
    U = Union

In [3]:
    with __import__('importnb').Notebook():
        try: from . import formatters
        except: import formatters

In [4]:
    class Display(IPython.display.DisplayHandle, traitlets.HasTraits):
        value = traitlets.Any()
        description = traitlets.Unicode(allow_none=True)
        
        def __init__(Display, value=None, description=None):
            IPython.display.DisplayHandle.__init__(Display)
            traitlets.HasTraits.__init__(Display, value=value, description=description)
        
        @traitlets.observe('value')
        def _observe_value(Display, change): Display.update(change['new'])
            
        def _ipython_display_(Display): 
            Display.description and IPython.display.display(IPython.display.Markdown(Display.description))
            Display.display(Display.value)

In [5]:
    class MultiDisplay:
        def __init__(self, *args): self.value = args
        def _ipython_display_(self):IPython.display.display(*self.value)

In [6]:
    def is_linkage(object): return isinstance(object, tuple) and (len(object) == 2) and isinstance(object[0], traitlets.HasTraits) and isinstance(object[1], str)

In [7]:
    class Links:
        """Initialize traitlet linkages for an interface."""
        def _display_key_value(Interface, key, value, context=None):
            if isinstance(value, str) and (Interface.parent.has_trait(value) or key == RTN): 
                display = Interface.parent, value
            elif isinstance(value, str) and Interface.has_trait(value): display = Interface, value
            else: display = Interface.display_formatter(value, context or {})
                
            (isinstance(display, Union) and Interface._link_union or
             isinstance(display, ipywidgets.Widget) and Interface._link_widget 
             or Interface._link_default)(key, value, display, context)
            return display
        
        def _link_default(Interface, key, value, display, context):
            if is_linkage(display):
                if not display[0].has_trait(display[1]): 
                    display[0].add_traits(**{display[1]: traitlets.Any()})
                container = getattr(display[0], '_container', {})
                Interface._container[key] = Display(container[display[1]] if display[1] in container else getattr(*display), key)
                if key == RTN: traitlets.dlink((Interface, key), display)
                else: traitlets.dlink(display, (Interface, key))
            else: Interface._container[key] = Display(value, key)
            traitlets.dlink((Interface, key), (Interface._container[key], 'value'))
            
        def _link_union(Interface, key, value, display, context):
            value = []
            for arg in display.__args__:
                if not isinstance(arg, type):
                    if isinstance(arg, str) and Interface.parent.has_trait(arg): arg = Interface.parent, arg
                    elif isinstance(arg, str) and Interface.has_trait(arg): arg = Interface, arg
                    value.append(Interface._display_key_value(key, arg, context=context))
                    if is_linkage(arg): 
                        traitlets.dlink(arg, (Interface, key))
                        traitlets.dlink((Interface, key), arg)
            Interface._container[key] = MultiDisplay(*value)

        def _link_widget(Interface, key, value, display, context):
            display.description = key
            Interface._container[key] = display
            if getattr(Interface, key) is not None: traitlets.dlink((Interface, key), (display, 'value'))
            traitlets.dlink((display, 'value'), (Interface, key))
            traitlets.dlink((Interface, key), (display, 'value'))                

        def add_linkages(Interface, **traits):
            context = Interface.ctx
            traits.pop(CTX, None)
            for key, value in traits.items():
                display = Interface._display_key_value(key, value, context)
                Interface.observe(Interface._update_object, key)
                
        def _default_linked_functions(Interface):
            for key in dir(Interface.object):
                value = getattr(Interface.object, key)
                if inspect.isfunction(value) and getattr(value, ANNO, None) and key[0].isalpha():
                    setattr(Interface, key, Function(value, parent=Interface))

In [8]:
    class ToSchema:
        """Schema inference for traits."""
        def to_schema(self, name=None, **object):
            annotations = dict(collections.ChainMap(self._implicit, self._explicit))
            annotations.pop(CTX, None)
            object.update({ANNO: annotations})
            for key in self.traits():
                if key[0].isalpha() and key not in dir(Interface):
                    if key not in object: object[key] = getattr(self.object, key, getattr(self, key))
                    if key not in annotations:
                        if key in getattr(self.object, ANNO, {}): annotations[key] = self.object.__annotations__[key]
            return type(name or '__main__', (__import__('pydantic').BaseModel,), object).schema()


In [9]:
    class Interface(traitlets.HasTraits, Links, ToSchema):
            
        shell = traitlets.Instance(IPython.InteractiveShell, help="A parent shell that will update object value post_execute.", allow_none=True)
        def _post_execute(Interface): [
            not isinstance(getattr(Interface, key), Function) and hasattr(Interface.object, key) and setattr(Interface, key, getattr(Interface.object, key)) 
            for key in Interface.traits()]

        parent = traitlets.Any(help="The parent interface provides links to external objects.")
        type = traitlets.Any()
        object = traitlets.Any()
        ctx = traitlets.Dict(allow_none=True)
        display_formatter = traitlets.Instance(IPython.core.formatters.BaseFormatter, allow_none=True)
        _container = traitlets.Dict(help="A named container of display objects for the annotations.")
        def __init__(Interface, value=None, **kwargs):
            kwargs['object'] = kwargs.get('object', value)
            super().__init__(**kwargs)
            Interface._init_shell()
            Interface._init_display_formatter()
            Interface.object = Interface.object or IPython.get_ipython().user_module
            Interface.parent = Interface.parent or Interface
            Interface.type = Interface.type or Interface.object
            Interface.add_traits() 
            
        def _init_shell(Interface):
            Interface.shell = Interface.shell or IPython.get_ipython()
            Interface.shell.events.register('post_execute', Interface._post_execute)
            
        def _init_display_formatter(Interface):
            Interface.display_formatter = formatters.RequestFormatter()
            Interface.display_formatter.for_type(object, lambda object: ipywidgets.interactive.widget_from_abbrev(object) or object)            
        def _ipython_display_(Interface): IPython.display.display(*Interface._container.values())
        def _update_object(Interface, change): setattr(Interface.object, change['name'], change['new'])
                
        @property
        def _explicit(Interface): return {k:v for k,v in getattr(Interface.type, ANNO, {}).items()}
        @property
        def _implicit(Interface): return {k:v for k,v in getattr(Interface.object, ANNO, {}).items()}
        
        def add_traits(Interface, **object):
            super().add_traits(**Interface._patch_non_traits(**object))
            with IPython.utils.capture.capture_output(stderr=False, stdout=False):
                Interface.add_linkages(**(object or collections.ChainMap(Interface._implicit, Interface._explicit)))
                Interface._default_linked_functions()
        
        def _patch_non_traits(Interface, **object):
            """Supplies traitlets for types basic on interactive and static types."""
            explicit, implicit = Interface._explicit, Interface._implicit
            Interface.ctx = dict(collections.ChainMap(implicit.pop(CTX, {}), explicit.pop(CTX, {}), getattr(Interface.parent, 'ctx', {}) ))
            traits = dict(object or collections.ChainMap(explicit, implicit))
            traits.pop(CTX, None)
            for key, value in traits.items():
                if not isinstance(value, traitlets.HasTraits):
                    default = getattr(Interface.object, key, getattr(Interface.type, key, None)) 
                    typ = explicit.get(key, implicit.get(key, None))
                    if isinstance(typ, (type, Union)):
                        union_types = [t for t in getattr(typ, '__args__', (typ,)) if isinstance(t, type)]
                        if union_types: traits[key] = traitlets.Union(tuple(traitlets.Instance(t) for t in union_types), default_value=default, allow_none=True)
                        else: traits[key]  =traitlets.Any(default)
                    else: traits[key] = traitlets.Any(default)   
            return traits


In [10]:
    class Function(Interface):
        def __call__(Function, change):
            value = Function.object(**{k: getattr(Function, k) for k in inspect.signature(Function.object).parameters})
            Function.has_trait(RTN) and setattr(Function, RTN, value)
            
        def __init__(Function, *args, **kwargs):
            super().__init__(*args, **kwargs)
            parameters = inspect.signature(Function.object).parameters
            for key, parameter in parameters.items():
                if getattr(Function, key) is None: setattr(Function, key, None if parameter.default is inspect._empty else parameter.default)
                Function.observe(Function, key)
            Function({'owner': Function})
            
        def add_traits(Function, **object):
            explicit, implicit = Function._explicit, Function._implicit
            object = dict(object or collections.ChainMap(explicit, implicit))
            object[RTN] = object.get(RTN, None)
            
            super().add_traits(**object)
            if isinstance(object[RTN], str):  
                object[RTN] = Function.parent, object[RTN]
            if is_linkage(object[RTN]): 
                display = object[RTN]
                display[0].has_trait(display[1]) or display[0].add_traits(**{display[1]: traitlets.Any()})
                traitlets.dlink((Function, RTN), display)
            
